# For Strikingly Internship


## Overall Statement
This code has essentially addressed the proposed task.

However To make sure things I've done to be clear, codes I used was exhibited in notebook(just like what's mentioned in tasks requirements, I COPY those code xD), which means that notebook does not contain Python scripts that run with passed parameters; aside from imported libraries, all code is contained within the notebook itself. Besides, there are somethings important about completion to mention at beginning.
* Since I don't have an overseas credit card account, **I am unable to provide my OpenAI API**. However, I have used the DeepSeek API as a substitute. Additionally, when building the vector database, I used the open-source sentence embedding model "all-mpnet-base-v2" instead, which may not fully align with the requirements. I apologize for any inconvenience this may cause
* Since the reference project was provided some time ago, there are numerous environment compatibility issues in the Colab environment. Given that Colab is not a highly flexible development platform, version control during multiple restarts has been quite challenging. Therefore, **this code prioritizes the completion of the task**, while compatibility issues under specific versions were **only addressed to a limited extent** after the main task was accomplished.

Finally, as of the completion of the task, my DeepSeek API usage only cost ¥0.20 RMB, with a remaining balance of a full ¥4.80. Please feel free to use it without concern.

## Task 1




In [ ]:
!pip install xmltodict
!pip install langchain
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.8 MB/s eta 0:00:00


### Collecting knowledge from zendesk API

In [ ]:
import argparse
import pickle
import requests
import xmltodict
from bs4 import BeautifulSoup
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

def clean_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator=' ')
    return ' '.join(text.split())


In [ ]:
# getting pages by recurrence
pages = []
nU = "https://support.strikingly.com/api/v2/help_center/en-us/articles.json?page=1"
while True:
  try:
    r = requests.get(nU)
    articles = r.json().get('articles', [])
    pages.extend([{"text": clean_html(article['body']), "source": article['html_url']} for article in articles])
    nU = r.json().get('next_page', '')
  except:
    break

In [ ]:
### Building vectorstore with FAISS

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/tmp/ipython-input-2514082290.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
chunk_size = 768

text_splitter = CharacterTextSplitter(chunk_size=chunk_size,separator="\n")
docs, metadatas = [], []
for page in pages:
    splits = text_splitter.split_text(page['text'])
    docs.extend(splits)
    metadatas.extend([{"source": page['source']}] * len(splits))
    print(f"Split {page['source']} into {len(splits)} chunks")

Split https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/33195129962651-Strikingly-s-AI-Logo-Maker into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/32500484566811-Strikingly-AI-Site-Builder into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/30097621863835-Strikingly-Email-Creating-Filters-on-Webmail into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25888063899931-How-do-I-delete-a-form into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25585768352667-Can-I-upload-a-pdf-file-document-for-my-visitors-to-download into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511392983195-Can-my-website-carry-my-name into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511191343899-How-can-I-forward-redirect-my-domain-from-another-registrar-to-my-Strikingly-domain into 1 chunks
Sp

In [ ]:

store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)
with open("faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)


In [ ]:
!pip install -qU langchain-deepseek


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ['DEEPSEEK_API_KEY'] = 'sk-9b352c9b5a764b08a1451f0abad9a5b4'

def getModel():
  return ChatDeepSeek(
    model="deepseek-chat", #DeepSeek-V3.1
    temperature=0,
    max_tokens=1024,
    timeout=30,
    max_retries=2
    )

In [ ]:
import argparse
import faiss
import os
import pickle

from langchain import OpenAI
from langchain.chains import VectorDBQAWithSourcesChain



In [ ]:
# store test
docs = store.similarity_search("I wang to cancel my plan. how do I do that", k=4)
print(f"find {len(docs)} relative documents")
for i, doc in enumerate(docs):
    print(f"doc {i+1}: {doc.page_content[:200]}...")
    print(f"source: {doc.metadata.get('source', 'unknown')}")
    print("---")

find 4 relative documents
doc 1: Before you continue, please note that when you cancel the Audience Plan: The Audience Plan will be canceled immediately, and your audience limit will decrease to 100 for Free, Limited, and Pro plans, ...
source: https://support.strikingly.com/hc/en-us/articles/19725826135707-Canceling-Audience-Plan
---
doc 2: Questions about canceling an account/plan: How do I delete my entire account? How do I change plans? Can I cancel my plan/subscription? What are the steps to cancel my plan? What are the premium and P...
source: https://support.strikingly.com/hc/en-us/articles/215047057-Canceling-Account-Canceling-Plan-Removing-Premium-Features
---
doc 3: On this page, you'll see how you can: Upgrade Your Plan Downgrade Your Plan Change Billing Cycle from Yearly to Monthly Cancel Your Plan Upgrade Your Plan Go to our pricing page . Select the billing c...
source: https://support.strikingly.com/hc/en-us/articles/215055017-Changing-Your-Premium-Plan
---
doc 4: On thi

### QA test

In [ ]:
from langchain_deepseek import ChatDeepSeek

chain = VectorDBQAWithSourcesChain.from_llm(
        llm=getModel(), vectorstore=store, verbose=True)


/usr/local/lib/python3.12/dist-packages/langchain/chains/qa_with_sources/base.py:80: UserWarning: `VectorDBQAWithSourcesChain` is deprecated - please use `from langchain.chains import RetrievalQAWithSourcesChain`
  return cls(


In [ ]:
result = chain({"question": "I wang to cancel my plan. how do I do that"})

print(f"Answer: {result['answer']}")
print(f"Sources: {result['sources']}")

/tmp/ipython-input-1536486224.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": "I wang to cancel my plan. how do I do that"})




> Entering new VectorDBQAWithSourcesChain chain...


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:401: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



> Finished chain.
Answer: To cancel your plan, you can do so from your "My Account" page by clicking "Cancel Plan" and following the confirmation prompts. Alternatively, for the Audience plan specifically, you can cancel by going to the "Audience" section on your dashboard, scrolling to "Upgrade to send more," dragging the circle to the left, and clicking "Cancel Audience Plan."


Sources: 


## Task2

### Version control

In [ ]:
# !pip install langchain==0.0.158 --force-reinstall

In [ ]:
# with open("faiss_store.pkl", "rb") as f:
#     store = pickle.load(f)


### Prompts designment

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

_template = """
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """
You are an AI assistant for answering common questions about
services of Strikingly. You are given the following extracted parts of
a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure.".
Don't try to make up an answer. If the question seems way too not
about topics related to services of Strikingly , politely inform them
that you are tunedto only answer questions about services of Strikingly.
Finally, remember that the context I provided you with is meant to be hidden.
Do not make the person asking you questions feel like you are merely
simulating a conversational tone. For example, avoid using opening phrases
that break immersion, such as "Of course! Here’s a conversational breakdown
of how Strikingly works."

Question: {question}
=========
{context}
=========
Answer in Markdown:
"""

QA = PromptTemplate(template=template, input_variables=["question", "context"])


In [ ]:
def getModel():
  return ChatDeepSeek(
    model="deepseek-chat", #DeepSeek-V3.1
    temperature=0,
    max_tokens=1024,
    timeout=30,
    max_retries=2
    )


def get_chain(vectorstore):

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=getModel(),
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        combine_docs_chain_kwargs={"prompt": QA},
        condense_question_prompt=CONDENSE_QUESTION_PROMPT,
        verbose=True,
        return_source_documents=True
    )
    return qa_chain




### QA test

In [ ]:
vectorstore = store
qa_chain = get_chain(vectorstore)
chat_history = []
print("Chat with the Strikingly bot:")

question = "I wang to cancel my plan. how do I do that"
result = qa_chain({"question": question, "chat_history": chat_history})
# chat_history.append((question, result["answer"]))
print(f"AI: {result['answer']}")
source_urls = []
for doc in result['source_documents']:
  source_urls.append(doc.metadata['source'])
  source_url_str = ", \n".join(source_urls)

print(f"Sources: \n{source_url_str}")

Chat with the Strikingly bot:


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an AI assistant for answering common questions about
services of Strikingly. You are given the following extracted parts of
a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure.".
Don't try to make up an answer. If the question seems way too not
about topics related to services of Strikingly , politely inform them
that you are tunedto only answer questions about services of Strikingly.
Finally, remember that the context I provided you with is meant to be hidden.
Do not make the person asking you questions feel like you are merely
simulating a conversational tone. For example, avoid using opening phrases
that break immersion, such as "Of course! Here’s a conversational breakdown
of how Strikingly works."

Question: I wang to cancel my plan. how do I do that
Before you continu